<a href="https://colab.research.google.com/github/lunaB/2020.AI.Sejong/blob/master/2020aidiscomfort_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 73%  
 공격성공(6/15-14:49)

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c 2020aidiscomfort

  0% 0.00/411 [00:00<?, ?B/s]
100% 411/411 [00:00<00:00, 359kB/s]
  0% 0.00/56.2k [00:00<?, ?B/s]
100% 56.2k/56.2k [00:00<00:00, 60.4MB/s]
  0% 0.00/2.39k [00:00<?, ?B/s]
100% 2.39k/2.39k [00:00<00:00, 2.39MB/s]


In [0]:
import numpy as np
import pandas as pd

In [0]:
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
import torch.nn.functional as F
import random
from sklearn import preprocessing

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

random.seed(777)
torch.manual_seed(777)

Scaler=preprocessing.StandardScaler()

* 데이터 로드 및 파싱

# 차별점 1 데이터 전처리
dropna 대신 값을 보간해서 사용함.
<pre>
xy_data['solar'] = xy_data['solar'].interpolate()
xy_data.fillna(value={'rain':0}, inplace=True)
</pre>

In [16]:
xy_data=pd.read_csv('train.csv')
print(xy_data.info())
print(xy_data.isna().sum())
xy_data=xy_data.fillna(0)
xy_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         1639 non-null   object 
 1   aveTemp      1639 non-null   float64
 2   maxTemp      1639 non-null   float64
 3   rain         655 non-null    float64
 4   aveHumidity  1639 non-null   float64
 5   solar        1620 non-null   float64
 6   Category     1639 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 89.8+ KB
None
date             0
aveTemp          0
maxTemp          0
rain           984
aveHumidity      0
solar           19
Category         0
dtype: int64


,date,aveTemp,maxTemp,rain,aveHumidity,solar,Category
0,2013-01-01,-4.7,0.0,3.1,73.5,5.76,0
1,2013-01-02,-11.7,-4.8,0.0,40.8,9.62,0
2,2013-01-03,-13.2,-9.3,0.0,42.8,9.78,0
3,2013-01-04,-10.7,-5.8,0.0,44.3,9.54,0
4,2013-01-05,-7.0,-1.2,0.0,59.4,8.09,0


In [19]:
xy_data.mean()['solar']

11.695654320987652

In [39]:
test_data=pd.read_csv('test.csv')
test_data

,date,aveTemp,maxTemp,rain,aveHumidity,solar
0,2019-06-06,21.2,25.1,7.5,75.9,5.33
1,2019-06-07,20.0,24.7,35.3,77.3,13.41
2,2019-06-09,21.6,26.5,2.5,66.1,16.87
3,2019-06-10,18.4,21.9,9.5,70.3,11.01
4,2019-06-15,21.9,27.4,2.1,61.5,25.94
...,...,...,...,...,...,...
63,2019-11-15,5.6,7.6,18.8,87.4,1.92
64,2019-11-17,8.3,10.5,21.1,79.4,1.32
65,2019-11-18,4.6,12.7,14.7,59.8,10.26
66,2019-11-19,0.2,4.0,0.0,48.0,11.93


In [42]:
import datetime as dt

xy_data=pd.read_csv('train.csv')
xy_data['solar'] = xy_data['solar'].interpolate()
xy_data.fillna(value={'rain':0}, inplace=True)

xy_data['date'] = pd.to_datetime(xy_data['date'])
xy_data['date'] = xy_data['date'].map(dt.datetime.toordinal)
xy_data

,date,aveTemp,maxTemp,rain,aveHumidity,solar,Category
0,734869,-4.7,0.0,3.1,73.5,5.76,0
1,734870,-11.7,-4.8,0.0,40.8,9.62,0
2,734871,-13.2,-9.3,0.0,42.8,9.78,0
3,734872,-10.7,-5.8,0.0,44.3,9.54,0
4,734873,-7.0,-1.2,0.0,59.4,8.09,0
...,...,...,...,...,...,...,...
1634,737055,-10.1,-6.5,0.0,28.3,10.44,0
1635,737056,-10.5,-6.6,0.0,33.0,10.36,0
1636,737057,-7.6,-3.0,0.0,29.0,10.57,0
1637,737058,-6.8,-0.9,0.0,35.9,10.48,0


In [44]:
test_data = pd.read_csv('test.csv')
test_data['solar'] = test_data['solar'].interpolate()
test_data.fillna(value={'rain':0}, inplace=True)

test_data['date'] = pd.to_datetime(test_data['date'])
test_data['date'] = test_data['date'].map(dt.datetime.toordinal)
test_data

,date,aveTemp,maxTemp,rain,aveHumidity,solar
0,737216,21.2,25.1,7.5,75.9,5.33
1,737217,20.0,24.7,35.3,77.3,13.41
2,737219,21.6,26.5,2.5,66.1,16.87
3,737220,18.4,21.9,9.5,70.3,11.01
4,737225,21.9,27.4,2.1,61.5,25.94
...,...,...,...,...,...,...
63,737378,5.6,7.6,18.8,87.4,1.92
64,737380,8.3,10.5,21.1,79.4,1.32
65,737381,4.6,12.7,14.7,59.8,10.26
66,737382,0.2,4.0,0.0,48.0,11.93


In [45]:
xy_data.isna().sum()

date           0
aveTemp        0
maxTemp        0
rain           0
aveHumidity    0
solar          0
Category       0
dtype: int64

In [46]:
test_data.isna().sum()

date           0
aveTemp        0
maxTemp        0
rain           0
aveHumidity    0
solar          0
dtype: int64

In [0]:
x_data=xy_data.loc[:,'aveTemp':'solar']
y_data=xy_data.loc[:,'Category']

In [0]:
x_data=np.array(x_data,dtype=float)
y_data=np.array(y_data,dtype=float)

x_data=Scaler.fit_transform(x_data)
x_train=torch.FloatTensor(x_data).to(device)
y_train=torch.LongTensor(y_data).to(device)

In [0]:
train_dataset=torch.utils.data.TensorDataset(x_train, y_train)

# 차별점2 (파라메터)
batch size가 너무 작음 (10 -> 100)

In [0]:
# 학습 파라미터 설정
learning_rate = 0.01
batch_size = 100

In [0]:
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

* 모델 설계

# 차별점3 (모델 세부)
학습데이터 변화에 따른 모델의 size 변경  
ReLU -> LeakyReLU

In [0]:
linear1=torch.nn.Linear(5,5,bias=True)
linear2=torch.nn.Linear(5,5,bias=True)
linear3=torch.nn.Linear(5,5,bias=True)
linear4=torch.nn.Linear(5,5,bias=True)
linear5=torch.nn.Linear(5,4,bias=True)
relu=torch.nn.LeakyReLU()

In [177]:
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[-0.6200, -0.4265,  0.1492,  0.6921,  0.6311],
        [ 0.2615, -0.1197,  0.1343,  0.2640,  0.3255],
        [ 0.5768,  0.5881, -0.8226, -0.4264, -0.6211],
        [-0.3756,  0.2849,  0.0681, -0.4451,  0.1572]], requires_grad=True)

In [0]:
model=torch.nn.Sequential(linear1,relu,
                          linear2,relu,
                          linear3,relu,
                          linear4,relu,
                          linear5
                          ).to(device)

In [0]:
loss = torch.nn.CrossEntropyLoss().to(device) # softmax 내부적으로 계산
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

* 모델 학습

In [180]:
total_batch = len(data_loader)
for epoch in range(300+1):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)
      
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    if epoch%100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 1.594089746
Epoch: 0101 cost = 0.528006375
Epoch: 0201 cost = 0.514517486
Epoch: 0301 cost = 0.509522140
Learning finished


* 모델 평가

In [0]:
with torch.no_grad():
    test=test_data.loc[:,'aveTemp':'solar']
    test=np.array(test,dtype=float)
    test=Scaler.transform(test)
    test=torch.from_numpy(test).float().to(device)
    prediction = model(test)
    correct_prediction = torch.argmax(prediction,dim=1)


* 결과값 제출

In [182]:
result=pd.read_csv('submit_sample.csv')
result

,Id,Category
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
63,63,0
64,64,0
65,65,0
66,66,0


In [183]:
for i in range(len(prediction)):
    result['Category'][i]=correct_prediction[i]
print(result)

    Id  Category
0    0         1
1    1         1
2    2         1
3    3         1
4    4         2
..  ..       ...
63  63         0
64  64         0
65  65         0
66  66         0
67  67         0

[68 rows x 2 columns]


In [0]:
result.to_csv('submission.csv',index=False)

In [185]:
!kaggle competitions submit -c 2020aidiscomfort -f 'submission.csv' -m "19010642_나영채"

100% 342/342 [00:03<00:00, 104B/s]
Successfully submitted to 2020.AI.Discomfort